# Air Quality Prediction Using Machine Learning

**Project by:** INLIGHN TECH  
**Platform:** Google Colab  
**Difficulty:** Medium  
**Technologies:** Python, scikit-learn, pandas, matplotlib, seaborn

---

## Project Description
This project focuses on predicting air quality using machine learning techniques. We will build a model to analyze and forecast air pollution levels based on historical data.

**Dataset Attributes:**
- `city`: City name
- `date`: Date of measurement
- `aqi`: Air Quality Index (Target variable)
- `co`: Carbon Monoxide (mg/m³)
- `no`: Nitric Oxide (µg/m³)
- `no2`: Nitrogen Dioxide (µg/m³)
- `o3`: Ozone (µg/m³)
- `so2`: Sulfur Dioxide (µg/m³)
- `pm2_5`: PM2.5 particles (µg/m³)
- `pm10`: PM10 particles (µg/m³)
- `nh3`: Ammonia (µg/m³)

## Step 1: Import Required Libraries

In [ ]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Set style for visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print("📊 Visualization style configured")

## Step 2: Data Generation (For Demonstration)

*Note: In a real project, you would load data from the UCI ML Repository or your own dataset file.*

In [ ]:
# Generate realistic air quality dataset for demonstration
# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("🏗️ CREATING AIR QUALITY DATASET")
print("=" * 40)

# Define major Indian cities
cities = ['Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Bangalore', 'Hyderabad', 
          'Ahmedabad', 'Pune', 'Surat', 'Jaipur', 'Lucknow', 'Kanpur']

# Generate date range (2 years of data)
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

print(f"📊 Generating dataset for {len(cities)} cities over {len(date_range)} days")

# Create realistic air quality data
data = []

for city in cities:
    for date in date_range:
        # Add seasonal and city-specific variations
        seasonal_factor = 1 + 0.3 * np.sin(2 * np.pi * date.timetuple().tm_yday / 365)
        
        # City pollution factors
        city_factors = {
            'Delhi': 1.8, 'Kanpur': 1.6, 'Lucknow': 1.4, 'Ahmedabad': 1.3,
            'Mumbai': 1.2, 'Kolkata': 1.3, 'Pune': 1.1, 'Jaipur': 1.2,
            'Chennai': 1.0, 'Bangalore': 0.9, 'Hyderabad': 1.0, 'Surat': 1.1
        }
        
        city_factor = city_factors.get(city, 1.0)
        combined_factor = seasonal_factor * city_factor
        
        # Generate correlated pollutant values
        row = {
            'city': city,
            'date': date.strftime('%Y-%m-%d'),
            'aqi': max(10, min(300, int(np.random.normal(80, 40) * combined_factor))),
            'co': round(max(0.1, np.random.exponential(2) * combined_factor), 2),
            'no': max(1, int(np.random.gamma(2, 15) * combined_factor)),
            'no2': max(5, int(np.random.gamma(2, 20) * combined_factor)),
            'o3': max(10, int(np.random.gamma(2, 25) * combined_factor)),
            'so2': max(1, int(np.random.gamma(2, 10) * combined_factor)),
            'pm2_5': max(5, int(np.random.gamma(2, 18) * combined_factor)),
            'pm10': max(10, int(np.random.gamma(2, 30) * combined_factor)),
            'nh3': max(1, int(np.random.gamma(2, 8) * combined_factor))
        }
        
        # Apply realistic limits
        row['aqi'] = max(10, min(300, row['aqi']))
        row['co'] = max(0.1, min(15.0, row['co']))
        row['no'] = max(1, min(200, row['no']))
        row['no2'] = max(5, min(150, row['no2']))
        row['o3'] = max(10, min(180, row['o3']))
        row['so2'] = max(1, min(100, row['so2']))
        row['pm2_5'] = max(5, min(150, row['pm2_5']))
        row['pm10'] = max(10, min(250, row['pm10']))
        row['nh3'] = max(1, min(50, row['nh3']))
        
        data.append(row)

# Create DataFrame
df = pd.DataFrame(data)

# Add some missing values (-200) as mentioned in requirements
missing_indices = np.random.choice(df.index, size=int(0.02 * len(df)), replace=False)
missing_columns = ['co', 'no2', 'pm2_5', 'pm10']

for idx in missing_indices:
    col = np.random.choice(missing_columns)
    df.at[idx, col] = -200

print(f"✅ Dataset created successfully!")
print(f"📊 Dataset shape: {df.shape}")
print(f"🏙️ Cities: {len(df['city'].unique())} cities")
print(f"📅 Date range: {df['date'].min()} to {df['date'].max()}")
print(f"❓ Missing values (-200): {(df == -200).sum().sum()} entries")

## Step 3: Data Understanding and Exploration

In [ ]:
# Display basic information about the dataset
print("=== DATASET OVERVIEW ===")
print(f"Dataset shape: {df.shape}")
print(f"\nColumn names: {list(df.columns)}")
print(f"\nData types:")
print(df.dtypes)

print("\n📋 First 10 rows:")
display(df.head(10))

print("\n📊 Statistical Summary:")
display(df.describe())

In [ ]:
# Check for missing values
print("=== MISSING VALUE ANALYSIS ===")

# Standard missing values (NaN)
print("\n🔍 Standard missing values (NaN):")
nan_missing = df.isnull().sum()
print(nan_missing[nan_missing > 0])

# Missing values marked as -200
print("\n❓ Missing values marked as -200:")
missing_200 = (df == -200).sum()
print(missing_200[missing_200 > 0])

# Cities and date range analysis
print("\n🏙️ Cities in dataset:")
print(f"   {', '.join(sorted(df['city'].unique()))}")
print(f"\n📅 Date range: {df['date'].min()} to {df['date'].max()}")

# Target variable analysis
print("\n🎯 Target Variable (AQI) Analysis:")
aqi_stats = df['aqi'].describe()
print(aqi_stats)

## Step 4: Data Visualization

In [ ]:
# Create visualizations for data exploration
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

# Plot distributions of key pollutants
pollutants = ['aqi', 'co', 'no2', 'pm2_5', 'pm10', 'o3']

for i, pollutant in enumerate(pollutants):
    # Filter out -200 values for visualization
    clean_data = df[df[pollutant] != -200][pollutant]
    
    axes[i].hist(clean_data, bins=30, alpha=0.7, edgecolor='black')
    axes[i].set_title(f'{pollutant.upper()} Distribution')
    axes[i].set_xlabel(pollutant)
    axes[i].set_ylabel('Frequency')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Air Quality Parameters Distribution', y=1.02, fontsize=16)
plt.show()

In [ ]:
# Missing values heatmap
plt.figure(figsize=(12, 8))
missing_data = (df == -200).astype(int)
sns.heatmap(missing_data.T, cbar=True, cmap='Reds', 
            xticklabels=False, yticklabels=df.columns)
plt.title('Missing Values Heatmap (Red = Missing)', fontsize=14)
plt.xlabel('Sample Index')
plt.ylabel('Features')
plt.show()

## Step 5: Data Preprocessing

In [ ]:
# Data preprocessing pipeline
print("=== DATA PREPROCESSING ===")

# Create a copy for processing
df_processed = df.copy()

# Step 1: Handle missing values (-200 → NaN → mean imputation)
print("\n🔧 Handling missing values...")
df_processed = df_processed.replace(-200, np.nan)

# Fill missing values with column mean for numeric columns
numeric_columns = df_processed.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    if df_processed[col].isnull().sum() > 0:
        mean_value = df_processed[col].mean()
        df_processed[col].fillna(mean_value, inplace=True)
        print(f"   ✅ Filled {col} missing values with mean: {mean_value:.2f}")

# Step 2: Convert date column to datetime
print("\n📅 Converting date column...")
df_processed['date'] = pd.to_datetime(df_processed['date'])
print("   ✅ Date column converted to datetime64[ns]")

print("\n✅ Data preprocessing completed!")
print(f"📊 Final dataset shape: {df_processed.shape}")
print(f"📊 Remaining null values: {df_processed.isnull().sum().sum()}")

## Step 6: Feature Engineering and Correlation Analysis

In [ ]:
# Feature engineering and correlation analysis
print("=== FEATURE ENGINEERING ===")

# Define features and target
feature_columns = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']
target_column = 'aqi'

print(f"🎯 Target variable: {target_column}")
print(f"🔢 Feature variables: {feature_columns}")

# Extract features and target
X = df_processed[feature_columns]
y = df_processed[target_column]

print(f"\n📊 Features shape: {X.shape}")
print(f"📊 Target shape: {y.shape}")

# Correlation analysis
print("\n🔗 Correlation with target variable (AQI):")
correlations = df_processed[feature_columns + [target_column]].corr()[target_column].sort_values(ascending=False)
for feature, corr in correlations.items():
    if feature != target_column:
        print(f"   {feature}: {corr:.3f}")

In [ ]:
# Correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df_processed[feature_columns + [target_column]].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.3f')
plt.title('Feature Correlation Matrix', fontsize=14)
plt.tight_layout()
plt.show()

## Step 7: Data Scaling and Train-Test Split

In [ ]:
# Feature scaling and train-test split
print("=== DATA SCALING AND SPLITTING ===")

# Scale features using StandardScaler
print("⚖️ Scaling features using StandardScaler...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("   ✅ Features scaled successfully")

# Split data into train and test sets
print("\n📊 Splitting data into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(f"   ✅ Training set: {X_train.shape[0]} samples")
print(f"   ✅ Testing set: {X_test.shape[0]} samples")
print(f"   ✅ Features: {X_train.shape[1]} dimensions")

print("\n✅ Data preparation completed!")

## Step 8: Model Training and Comparison

In [ ]:
# Define model evaluation function
def evaluate_model(y_true, y_pred, model_name):
    """Calculate and display model evaluation metrics"""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n📊 {model_name} Results:")
    print(f"   Mean Absolute Error (MAE): {mae:.4f}")
    print(f"   Root Mean Square Error (RMSE): {rmse:.4f}")
    print(f"   R-squared (R²): {r2:.4f}")
    
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

In [ ]:
# Train and evaluate multiple models
print("🤖 TRAINING MULTIPLE MODELS")
print("=" * 40)

models = {}
results = {}
predictions = {}

# 1. Random Forest Regressor
print("\n🌲 Training Random Forest Regressor...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

models['Random Forest'] = rf_model
predictions['Random Forest'] = rf_pred
results['Random Forest'] = evaluate_model(y_test, rf_pred, 'Random Forest')

# 2. Linear Regression
print("\n📈 Training Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)

models['Linear Regression'] = lr_model
predictions['Linear Regression'] = lr_pred
results['Linear Regression'] = evaluate_model(y_test, lr_pred, 'Linear Regression')

# 3. Support Vector Regressor
print("\n🎯 Training Support Vector Regressor...")
svr_model = SVR(kernel='rbf', C=1.0, gamma='scale')
svr_model.fit(X_train, y_train)
svr_pred = svr_model.predict(X_test)

models['SVR'] = svr_model
predictions['SVR'] = svr_pred
results['SVR'] = evaluate_model(y_test, svr_pred, 'SVR')

In [ ]:
# Model comparison
print("\n📊 MODEL COMPARISON RESULTS")
print("=" * 50)

# Create comparison DataFrame
comparison_df = pd.DataFrame(results).T
comparison_df = comparison_df.round(4)
display(comparison_df)

# Find best model
best_model_name = comparison_df['R2'].idxmax()
best_r2_score = comparison_df.loc[best_model_name, 'R2']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   R² Score: {best_r2_score:.4f}")
print(f"   MAE: {comparison_df.loc[best_model_name, 'MAE']:.4f}")
print(f"   RMSE: {comparison_df.loc[best_model_name, 'RMSE']:.4f}")

## Step 9: Feature Importance Analysis

In [ ]:
# Feature importance analysis (Random Forest)
print("🌟 FEATURE IMPORTANCE ANALYSIS")
print("=" * 35)

# Get feature importance from Random Forest
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nRandom Forest Feature Importance:")
for _, row in feature_importance.iterrows():
    print(f"   {row['feature']}: {row['importance']:.3f}")

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature', palette='viridis')
plt.title('Random Forest Feature Importance', fontsize=14)
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.tight_layout()
plt.show()

## Step 10: Model Performance Visualization

In [ ]:
# Model performance comparison visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Model R² comparison
models_list = list(results.keys())
r2_scores = [results[model]['R2'] for model in models_list]

bars = axes[0].bar(models_list, r2_scores, color=['skyblue', 'lightgreen', 'coral'])
axes[0].set_title('Model Performance Comparison (R² Score)', fontsize=14)
axes[0].set_ylabel('R² Score')
axes[0].set_ylim(0, max(r2_scores) * 1.1)

# Add value labels on bars
for bar, score in zip(bars, r2_scores):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                f'{score:.3f}', ha='center', va='bottom')

# Plot 2: Predicted vs Actual (best model)
best_pred = predictions[best_model_name]
axes[1].scatter(y_test, best_pred, alpha=0.6, color='blue')

# Perfect prediction line
min_val = min(y_test.min(), best_pred.min())
max_val = max(y_test.max(), best_pred.max())
axes[1].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')

axes[1].set_xlabel('Actual AQI')
axes[1].set_ylabel('Predicted AQI')
axes[1].set_title(f'{best_model_name} - Predicted vs Actual', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 11: Residual Analysis

In [ ]:
# Residual analysis for the best model
print(f"📊 RESIDUAL ANALYSIS - {best_model_name}")
print("=" * 40)

best_pred = predictions[best_model_name]
residuals = y_test - best_pred

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Residual plot
axes[0].scatter(best_pred, residuals, alpha=0.6)
axes[0].axhline(y=0, color='red', linestyle='--')
axes[0].set_xlabel('Predicted AQI')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residual Plot', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Residual distribution
axes[1].hist(residuals, bins=30, alpha=0.7, edgecolor='black')
axes[1].axvline(residuals.mean(), color='red', linestyle='--', 
               label=f'Mean: {residuals.mean():.2f}')
axes[1].set_xlabel('Residuals')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Residual Distribution', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Residual statistics
print(f"\nResidual Statistics:")
print(f"   Mean: {residuals.mean():.4f}")
print(f"   Std Dev: {residuals.std():.4f}")
print(f"   Min: {residuals.min():.4f}")
print(f"   Max: {residuals.max():.4f}")

## Step 12: Save Results and Export Data

In [ ]:
# Save results and create output files
print("💾 SAVING RESULTS")
print("=" * 20)

# Save processed dataset
df_processed.to_csv('processed_air_quality_data.csv', index=False)
print("✅ Processed dataset saved: 'processed_air_quality_data.csv'")

# Save model comparison results
comparison_df.to_csv('model_comparison_results.csv')
print("✅ Model comparison saved: 'model_comparison_results.csv'")

# Save feature importance
feature_importance.to_csv('feature_importance.csv', index=False)
print("✅ Feature importance saved: 'feature_importance.csv'")

# Save predictions
predictions_df = pd.DataFrame({
    'actual_aqi': y_test.values,
    'predicted_aqi': best_pred,
    'residual': residuals
})
predictions_df.to_csv('aqi_predictions.csv', index=False)
print("✅ Predictions saved: 'aqi_predictions.csv'")

print("\n📊 All results saved successfully!")

## Step 13: Project Summary and Conclusions

In [ ]:
# Final project summary
print("🎉 PROJECT SUMMARY")
print("=" * 30)

print(f"📂 Dataset: Air Quality dataset with {len(df_processed):,} samples")
print(f"🏙️ Cities: {len(df_processed['city'].unique())} cities")
print(f"📅 Time Period: 2022-2023 (2 years)")
print(f"🎯 Target: Air Quality Index (AQI)")
print(f"🔢 Features: {len(feature_columns)} pollutant parameters")
print(f"🏆 Best Model: {best_model_name} (R² = {best_r2_score:.4f})")
print(f"📈 Performance: MAE = {comparison_df.loc[best_model_name, 'MAE']:.2f}, RMSE = {comparison_df.loc[best_model_name, 'RMSE']:.2f}")
print(f"🌟 Most Important Feature: {feature_importance.iloc[0]['feature']} ({feature_importance.iloc[0]['importance']:.3f})")

print("\n💡 KEY FINDINGS:")
findings = [
    f"• {best_model_name} performed best with R² = {best_r2_score:.4f}",
    f"• {feature_importance.iloc[0]['feature'].upper()} is the most predictive feature",
    f"• All pollutants show moderate correlation with AQI",
    f"• Model achieved MAE of {comparison_df.loc[best_model_name, 'MAE']:.1f} AQI units",
    "• Missing values were successfully handled with mean imputation"
]

for finding in findings:
    print(f"   {finding}")

print("\n🎯 RECOMMENDATIONS:")
recommendations = [
    "• Include meteorological data for better predictions",
    "• Implement time series analysis for temporal patterns",
    "• Consider ensemble methods for improved accuracy",
    "• Validate with real UCI ML Repository dataset",
    "• Deploy for real-time air quality monitoring"
]

for rec in recommendations:
    print(f"   {rec}")

print("\n✅ AIR QUALITY PREDICTION PROJECT COMPLETED SUCCESSFULLY!")
print("📝 All INLIGHN TECH requirements have been implemented!")